# ``Easy``

In [ ]:
import sympy as sp
import matplotlib.pyplot as plt
import pySecDec as psd # Import pySecDec

In this notebook, we experiment with computing a simple dimensionally regulated integral:

$$
  I[c] \equiv \int_0^1 \mathrm{d}x \int_0^1 \mathrm{d} y\ (c x+y)^{-2+\epsilon}
$$

### Generating an integral library

We will use the ``make_package`` function which generates a c++ package for evaluating the integral.

First, we give our integral a name ``easy`` and we tell pySecDec the integration variables, any real parameters the integral depends on, the dimensional regulators, the order to which we want to know the result (as a series expansion in the regulators), and the input polynomial.

In [ ]:
! rm -rf easy
package = psd.make_package(
    name = 'easy',
    integration_variables = ['x','y'],
    real_parameters = ['c'],
    regulators = ['eps'],
    requested_orders = [0],
    polynomials_to_decompose = ['(c*x+y)^(-2+eps)']
)

Next, we build the integral library, ready for numerical integration.

In [ ]:
! make -C easy > /dev/null 2>&1

### Calling the integral library and manipulating the result

We now have an integral library ``easy/easy_pylink.so`` which we can load into python and call to evaluate our integral.

In [ ]:
easy = psd.integral_interface.IntegralLibrary('easy/easy_pylink.so')

The library will return: 
* the integral result without prefactors, 
* the prefactor
* the integral result with prefactors. 

Usually, we only care about the result with the prefactor, so we will use only that part of the output. In this case the prefactor is also just ``1``.

We begin by evaluating our integral at the point ``c=0.1``.

In [ ]:
c=0.1
str_integral_without_prefactor, str_prefactor, str_integral_with_prefactor = easy([c])
print(str_integral_with_prefactor)

The result returned by our library is just a string encoding the full result, it depends on all regulators and also tells us the numerical uncertainy on the result. We have convenience functions for converting the output to a format compatible with various symbolic libraries. Here we will use the sympy output format.

In [ ]:
result_sympy = psd.integral_interface.series_to_sympy(str_integral_with_prefactor)

In [ ]:
print('res:', result_sympy[0]) # Result
print('err:', result_sympy[1]) # Numerical uncertainty 

We can pick out the finite part of the result using sympy's ``Poly`` module:
* First we strip the trailing ``+ O(eps)`` (Big-O notation) from the string.
* Next, we convert the result to a sympy ``Poly``.
* Finally we can ask for the last coefficient (i.e. the finite part if ``requested orders=0``).

In [ ]:
if result_sympy[0].endswith('+ O(eps)'):
    result_sympy_stripped = result_sympy[0][:-8]
result_finite_part = sp.Poly(result_sympy_stripped).coeffs()[-1]
print(result_finite_part)

### Plotting the result for different values of the real_parameters

We can also numerically compute our integral at several points, by repeatedly calling the library.

In [ ]:
xx = []
yy = []
for cc in np.arange(0.1,0.9,0.05):
    _, _, result = easy([cc])
    xx.append(cc)
    yy.append(sp.Poly(psd.integral_interface.series_to_sympy(result)[0][:-8]).coeffs()[1])

In [ ]:
plt.plot(xx,yy,marker='o')